In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
import math
import torch
from torch import nn
import random
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, GPTNeoXForCausalLM
from huggingface_hub import snapshot_download

for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

sys.path.append("/nas/ucb/henrypapadatos/Open-Assistant/model")
import model_training.models.reward_model



NVIDIA RTX A6000


In [2]:
# file_name = 'Bo32_OpenAssistant_pythia-12b-sft-v8-2.5k-steps_11_16_2023_21_32_18_feedback_poem_300.csv'
# file_name = 'Bo32_openchat_openchat_3.5_11_28_2023_12_28_40_feedback_poem_5.csv'
file_name = 'Bo32_openchat_openchat_3.5_11_28_2023_15_18_45_feedback_poem_300.csv'
file_path = os.path.join('results', file_name)
df = pd.read_csv(file_path)

In [3]:
# model_name_or_path = "OpenAssistant/oasst-rm-2-pythia-6.9b-epoch-1"
model_name_or_path = 'Starling'

#this code is copy pasted from Starling
class GPTRewardModel(nn.Module):
    def __init__(self, model_path):
        super().__init__()
        model = AutoModelForCausalLM.from_pretrained(model_path, cache_dir='/scratch/henrypapadatos', device_map="auto")
        self.config = model.config
        self.config.n_embd = self.config.hidden_size if hasattr(self.config, "hidden_size") else self.config.n_embd
        self.model = model
        self.transformer = model.model
        self.v_head = nn.Linear(self.config.n_embd, 1, bias=False).to(self.model.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.tokenizer.pad_token = self.tokenizer.unk_token
        self.PAD_ID = self.tokenizer(self.tokenizer.pad_token)["input_ids"][0]

    def get_device(self):
        return self.model.device

    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        position_ids=None,
    ):
        """
        input_ids, attention_mask: torch.Size([bs, seq_len])
        return: scores: List[bs]
        """
        bs = input_ids.shape[0]
        transformer_outputs = self.transformer(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            position_ids=position_ids,
        )
        hidden_states = transformer_outputs[0]
        scores = []
        rewards = self.v_head(hidden_states).squeeze(-1)
        for i in range(bs):
            c_inds = (input_ids[i] == self.PAD_ID).nonzero()
            c_ind = c_inds[0].item() if len(c_inds) > 0 else input_ids.shape[1]
            scores.append(rewards[i, c_ind - 1])
        return scores

if 'OpenAssistant' in model_name_or_path:

    model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side="left", legacy=False)

elif 'Starling' in model_name_or_path:

    #this code is copy pasted from Starling
    ## Load the model and tokenizer
    reward_model = GPTRewardModel("meta-llama/Llama-2-7b-chat-hf")
    reward_tokenizer = reward_model.tokenizer
    reward_tokenizer.truncation_side = "left"
    reward_device = reward_model.get_device()
    reward_batch_size = 1
    directory = snapshot_download("berkeley-nest/Starling-RM-7B-alpha", cache_dir='/scratch/henrypapadatos')
    for fpath in os.listdir(directory):
        if fpath.endswith(".pt") or fpath.endswith("model.bin"):
            checkpoint = os.path.join(directory, fpath)
            break

    # Load the model on the GPU  
    # reward_model.load_state_dict(torch.load(checkpoint, map_location=reward_device), strict=False)
    reward_model.load_state_dict(torch.load(checkpoint), strict=False)
    reward_model.eval().requires_grad_(False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
def get_reward_openassistant(input):
    inputs = tokenizer(input, return_tensors="pt").to(model.device)
    score = model(**inputs).logits[0].cpu().detach().item()    
    return score

def get_reward_openchat(input):

    reward_batch_size = 1
    #modify the tags to make it fit Llama2
    input = input.replace('GPT4 Correct User:', '[INST]')
    input = input.replace('GPT4 Correct Assistant:', '[/INST]')
    input = input.replace('<|end_of_turn|>', '</s>')

    #This code is copy pasted from Starling
    """samples: List[str]"""
    samples = [input]
    input_ids = []
    attention_masks = []
    encodings_dict = reward_tokenizer(
        samples,
        truncation=True,
        max_length=2048,
        padding="max_length",
        return_tensors="pt",
    ).to(reward_device)
    input_ids = encodings_dict["input_ids"]
    attention_masks = encodings_dict["attention_mask"]
    mbs = reward_batch_size
    out = []
    for i in range(math.ceil(len(samples) / mbs)):
        rewards = reward_model(input_ids=input_ids[i * mbs : (i + 1) * mbs], attention_mask=attention_masks[i * mbs : (i + 1) * mbs])
        out.extend(rewards)
    score = torch.hstack(out).cpu().detach().item()
    # print('score: ', score)
    return score

In [5]:
#take the N_responses column
N_response = df['N_response']

#create a dictionary with keys 1,2,4,8,16,32; these will be the number of samples (N)
BoN = {}
for i in range(6):
    BoN[2**i] = []

#iterate through the N_responses column and for each response
for [index, response] in tqdm(N_response.items()):
    #response is a string, we convert it to a list
    response_list = ast.literal_eval(response)
    score_list = []
    #iterate over each of the 32 generated responses for each prompt and get the reward score
    for el in response_list:
        if 'openchat' or 'Starling' in model_name_or_path:
            score = get_reward_openchat(el)
        elif 'OpenAssistant' in model_name_or_path:
            score = get_reward_openassistant(el)
        else: 
            print('Error: file name not recognized')
        score_list.append(score)
    
    #we now sample N(=key) elements from the score_list and store the best response in the BoN dictionary
    for key in BoN.keys():
        random_scores = random.sample(score_list, key)
        best_score_index = score_list.index(max(random_scores))
        #we keep inly the response, not the prompt and we remove the eos token
        if 'openchat' or 'Starling' in model_name_or_path:
            best_response = response_list[best_score_index].split('GPT4 Correct Assistant: Here are my thoughts on the poem:')[1].replace('<|end_of_turn|>','')
        elif 'OpenAssistant' in model_name_or_path:                                                        
            best_response = response_list[best_score_index].split('<|assistant|>Here are my comments:')[1].replace('<|endoftext|>','')
        # print('Best response for N = ', key)     
        # print(best_response)
        # print('########################################################################')
        BoN[key].append(best_response)

180it [2:52:22, 57.46s/it]


In [7]:
#add the dictionary to the dataframe
for key in BoN.keys():
    df['BoN_'+str(key)] = BoN[key]

new_file_name = file_name[:-4] + '_' + model_name_or_path + '.csv'
new_file_name = new_file_name.replace('/', '_')
print(new_file_name)
new_file_path = os.path.join('results', new_file_name)

#save dataframe as csv
df.to_csv(new_file_path, index=False)

df.head()

Bo32_openchat_openchat_3.5_11_28_2023_15_18_45_feedback_poem_300_Starling.csv


,prompt,N_response,type,BoN_1,BoN_2,BoN_4,BoN_8,BoN_16,BoN_32
0,GPT4 Correct User:Please comment briefly on th...,['<s> GPT4 Correct User:Please comment briefly...,base,\n\nThe poem is an exploration of the interpla...,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem is a rich exploration of the them...,\n\nThe poem is a beautiful and contemplative ...,\n\nThe poem is a rich and evocative explorati...,\n\nThe poem is a rich and evocative explorati...
1,GPT4 Correct User:Please comment briefly on th...,['<s> GPT4 Correct User:Please comment briefly...,like,\n\nThe poem is a beautiful and deeply philoso...,\n\nThe poem is a captivating exploration of b...,\n\n1. The poet presents a stunning imagery of...,\n\nThe poem is a beautiful and thought-provok...,\n\nThis poem is a beautiful exploration of th...,\n\nThis poem is a beautiful exploration of th...
2,GPT4 Correct User:Please comment briefly on th...,['<s> GPT4 Correct User:Please comment briefly...,dislike,\n\nThe poem explores the themes of cosmic ser...,\n\nThe poem explores the relationship between...,"\n\nThe poem has an ambitious scope, attemptin...",\n\n1. The poem is rich in imagery and metapho...,\n\nThe poem is a beautiful exploration of the...,\n\nThe poem is a beautiful exploration of the...
3,GPT4 Correct User:Please comment briefly on th...,['<s> GPT4 Correct User:Please comment briefly...,base,\n\nThe poem is a powerful and impassioned cry...,\n\nThe poem is a powerful indictment of injus...,\n\nThe poem is a powerful indictment of injus...,\n\nThe poem beautifully captures the essence ...,\n\nThe poem is a powerful and evocative portr...,\n\nThe poem is a powerful and evocative portr...
4,GPT4 Correct User:Please comment briefly on th...,['<s> GPT4 Correct User:Please comment briefly...,like,\n\nThe poem is a powerful and evocative explo...,\n\nThe poem is a powerful indictment of injus...,\n\nThe poem is a powerful commentary on the t...,\n\nThe poem is a powerful exploration of the ...,\n\nThe poem is a powerful and evocative exami...,\n\nThe poem is a powerful commentary on the t...
